In [ ]:
decimation_version = 0
decimation_ratio = 0.25
import time


@schema
class Martinotti_Attributes(dj.Computed):
    definition="""
    -> minnie.Decimation #segment ids and mesh data for all decimated segments
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ---
    centroid_x           : int unsigned                 # (EM voxels)
    centroid_y           : int unsigned                 # (EM voxels)
    centroid_z           : int unsigned                 # (EM voxels)
    distance_from_prediction : double                   # the distance of the ALLEN predicted centroid soma center from the algorithms prediction
    soma_vertices             : longblob                # array of vertices
    soma_faces             : longblob                   # array of faces
    multiplicity         : tinyint unsigned             # the number of somas found for this base segment
    run_time : double                   # the amount of time to run (seconds)

    """
    
    #further restrict table to include just subset
    key_source = (minnie.Decimation & (dj.U("segment_id") & (m65.AllenSegmentCentroid & "status=1").proj()) & "version=" + str(decimation_version))
    
    # MAKE IS WHAT CALCULATES THE ATTRIBUTES AND INSERTS IT INTO YOUR TABLE
    def make(self,key):
        
        # Get the Mesh Data
        print(f"\n\n\n---- Working on {key['segment_id']} ----")
        
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
        
        segment_id = key["segment_id"]
        
        # Extract Soma Center
        total_soma_list, run_time = extract_soma_center(segment_id,current_mesh_verts,current_mesh_faces)
        print(f"Run time was {run_time}  and the total_soma_list = {total_soma_list} and ")
        
        # IF NO SOMA
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            insert_dict = dict(key,
                              soma_index=-1,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               distance_from_prediction=None,
                               soma_vertices=None,
                               soma_faces=None,
                               multiplicity=0,
                               run_time=run_time
                              )
            
            raise Exception("to prevent writing because none were found")
            # Store the attributes that you found in a dictionary
            # Incert the row of attributes for that key
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        # If >0 Soma
        
        #get the soma prediction
        c_x,c_y,c_z = (m65.AllenSegmentCentroid() & key).fetch1("centroid_x","centroid_y","centroid_z")
        allen_centroid_prediction = np.array([c_x,c_y,c_z])
        
        dicts_to_insert = []
        
        # If >1 Soma
        if len(total_soma_list) > 1:
            raise Exception("to prevent writing MULTILPLE SOMAS TO DATABASE")
            
        for i,current_soma in enumerate(total_soma_list):
            print("Trying to write off file")
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            auto_prediction_center = np.mean(current_soma.vertices,axis=0)
            
            error_distance = np.linalg.norm(allen_centroid_prediction-auto_prediction_center)
            
            insert_dict = dict(key,
                              soma_index=i,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
                               distance_from_prediction=error_distance,
                               soma_vertices=current_soma.vertices,
                               soma_faces=current_soma.faces,
                               multiplicity=len(total_soma_list),
                               run_time=run_time
                              )
            
            
            
            dicts_to_insert.append(insert_dict)
            
        #raise Exception("to prevent writing")
            
        self.insert(dicts_to_insert,skip_duplicates=True)

In [ ]:
"""
Actually will do the population
"""

#(schema.jobs & "table_name='__whole_auto_annotations_label_clusters3'")#.delete()
dj.config["enable_python_native_blobs"] = True

import time
start_time = time.time()
Martinotti_Attributes.populate(reserve_jobs=True) # runs parallel
print(f"Total time for SomaCentroidValidation populate = {time.time() - start_time}")